In [30]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import html5lib
from transformers import pipeline
import tensorflow as tf

In [3]:
def get_source(url):
    """Return the source code of the provided url.
    
    Args: 
        url (string): URL of the page to scrape.
        
    Returns:
        response (object): HTTP response object from requests_html
    """  
    
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    
    except requests.exceptions.RequestException as e:
        print(e)

In [4]:
def scrape_google(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    response1 = get_source("https://www.google.com/search?q=" + query + '&start=10')
    
    links = list(response.html.absolute_links)
    links += list(response1.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    
    return links

In [5]:
links = scrape_google("make worklife healthier")

In [15]:
def get_body(url):
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    coverpage_data = soup1.find_all('p')
    heading1 = soup1.find_all('h1')[0].get_text()
    heading2 = soup1.find_all('h2')
    heading2 = [heading2[i].get_text() for i in range(len(heading2))]
    data = [coverpage_data[i].get_text() for i in range(len(coverpage_data))]
    
    if len(heading1) > 0:

        headings = heading1 + " ".join(heading2)
    elif len(heading1) < 1:
        headings = " ".join(heading2)
    else:
        #Run the machine learning model here ML
        pass
    final_text = " ".join(data)
    final_text = final_text.replace('.', '.<eos>')
    final_text = final_text.replace('?', '.<eos>')
    final_text = final_text.replace('!', '.<eos>')
    final_text = final_text.split('<eos>')
    return [headings, final_text]

In [16]:
def add_data_to_df(links):
    all_headings_text = []
    for i in links:
        curr = get_body(i)
        all_headings_text.append(curr)
        
    df = pd.DataFrame(all_headings_text, columns = ['Headings', 'Text'])
    return df

In [17]:
df = add_data_to_df(links)

In [18]:
df

,Headings,Text
0,16 Best Tips To Maintain a Healthy Work-Life B...,"[Today, quality of work-life balance is one of..."
1,How to have a good work-life balance (hint: it...,[Products Build leaders that accelerate team p...
2,How to Improve Your Work-Life Balance TodayBal...,"[Often, work takes precedence over everything ..."
3,6 Tips For Better Work-Life Balance,[This article is part of a series on work-life...
4,12 Steps to Finding Your Ideal Work-Life Balan...,"[Thanks to remote work, super-smart tech, and ..."
5,\n\t\t\t\t\t\t\t\t15 tips to create a healthy ...,[This is having devastating consequences on ou...
6,The Importance of Work-Life BalanceWork-Life B...,[Work-life balance has been in the news a lot ...
7,25 Best Ways To Achieve Work-Life Balance and ...,[ Vantage Rewards Vantage Pulse Vantage P...
8,\nWork-life balance\n\nHeader menu Header menu...,[A healthy work-life balance will mean differe...
9,Try these 12 tips to improve your work-life ba...,[Products Build leaders that accelerate team p...


In [21]:
df['Headings'][0]

"16 Best Tips To Maintain a Healthy Work-Life Balance1. 'Me' time 2. Get social 3. Learn to prioritize 4. Talk to your boss 5. Stay organized 6. Healthy eating habits 7. Workout 8.   Take vacations 9.   Take small breaks 10. Learn to say 'No' 11. Digital Detox 12. Proper sleep 13. Meditate 14. Do not compare 15. Find work that suits your needs 16. Set realistic expectations People Also viewed 👀 What Is Career Planning & How to Do It 6 Interview Mistakes to Avoid in 2023 Tips to Rejoining the Workforce in 2023 7 Promising Career Options to Pursue in 2023 6 Ways to Find Your First Job in 2023 6 Trending Courses to Sign Up for in 2023"

In [31]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.